In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
from konlpy.tag import Okt
# 텍스트 데이터를 머신러닝 알고리즘이 처리할 수 있는 수치 벡터로 변환
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel
from tqdm.auto import tqdm
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

In [25]:
df = pd.read_csv("C:/Users/ezen602-09/total.csv", encoding="utf-8-sig")

In [24]:
# 추가할 새 데이터 
new_data = [
    # 폭행 / 부당한 대우
    {'input': '남편의 폭력 때문에 더는 못 살겠어요.', 'is_divorce': 1},
    {'input': '아내의 계속되는 폭언 때문에 정신적으로 너무 힘듭니다.', 'is_divorce': 1},
    {'input': '배우자가 저를 무시하고 인격적으로 모독하는데, 헤어지고 싶어요.', 'is_divorce': 1},
    {'input': '지속적인 가정폭력으로 신변에 위협을 느껴 이혼을 결심했습니다.', 'is_divorce': 1},
    
    # 외도 / 부정행위
    {'input': '배우자가 바람피운 사실을 알게 됐습니다. 관계를 끝내고 싶어요.', 'is_divorce': 1},
    {'input': '상간녀 소송을 준비하면서 이혼도 같이 진행하려고요.', 'is_divorce': 1},
    
    # 경제적 문제
    {'input': '남편이 생활비를 전혀 주지 않아 이혼을 생각 중입니다.', 'is_divorce': 1},
    {'input': '아내가 저 몰래 큰 빚을 졌는데, 감당이 안 됩니다.', 'is_divorce': 1},
    {'input': '배우자가 도박에 빠져서 가산을 탕진했어요.', 'is_divorce': 1},
    
    # 기타 사유
    {'input': '성격 차이가 너무 심해서 헤어지기로 마음먹었어요.', 'is_divorce': 1},
    {'input': '시댁과의 갈등으로 혼인 관계를 유지하기 힘듭니다.', 'is_divorce': 1},
    {'input': '별거한 지 오래되었고, 이제 서류 정리를 하고 싶어요.', 'is_divorce': 1},
]
df_new = pd.DataFrame(new_data)
df_updated = pd.concat([df, df_new], ignore_index=True)
df_updated.to_csv("C:/Users/ezen602-09/total.csv", index=False, encoding='utf-8-sig')

print(f"\n새롭게 추가된 데이터 개수: {len(df_new)}개")
print(f"총 데이터 개수: {len(df_updated)}개")
print("이 새로운 파일을 사용하여 모델을 다시 학습시켜 보세요!")


새롭게 추가된 데이터 개수: 12개
총 데이터 개수: 4071개
이 새로운 파일을 사용하여 모델을 다시 학습시켜 보세요!


# KoNLPy를 사용한 정교한 전처리

In [27]:
MINIMAL_STOPWORDS = [
    '것', '수', '때', '등', '들', '더', '이', '그', '저', '나', 
    '우리', '같', '또', '만', '년', '월', '일', '하다', '있다', '되다', '가하다','이루어지다',
     '가능하다', '가능', '가다', '되다', '하다', '있다', '없다', '않다', '된다', '한다',
    '어떻게', '어떤', '무엇', '언제', '어디서', '왜', '누가', '얼마', '몇',
    '알고', '싶다', '궁금하다', '문의', '질문', '답변', '설명', '이해',
    '과정', '절차', '이후', '다음', '먼저', '그리고', '그러나', '하지만', '그래서', '제자','제호','제조',

     '없', '있', '하', '되', '않', 
    '나', '우리', '너', '당신', '같', '또', '것', '때', '등', 
    '때문', '정도', '사실', '생각', '경우', '문제', '방법', '상황', '내용', '결과', '사람',

    '해야', '하면', '경우', '때는', '어느', '무슨', '어디', '누구' , ' 가지다' , '가지', '하나요', '위해', '이혼',
     '조건', '대한', '관련', '따르다', '판단', '인정', '적용', 
    '효력', '성립', '발생', '요건', '증명', '근거', '주장'
]


# # 법률 전문용어는 무조건 포함
LEGAL_KEYWORDS = [ '위자료', '재산분할', '양육권', '친권', '면접교섭', 
    '협의이혼', '청구', '배상', '손해', '책임',
    '차용금', '반환', '취소', '원상회복', '사해행위', '채권자', '배우자', '이혼사유',
     '이혼', '사유', 
    '혼인', '금전거래', '청구권',  '액수', '정해지', '부적법',
    '혼인파탄', '파탄', '분할', '양육비', '면접',
    '교섭', '협의', '조정신청', '손해배상', '부부', '배우자', '당사자', '사람', '개인', '상대방', 
]


def preprocess_text(text):  # 전처리 함수 
    """
    한국어 텍스트를 입력받아 전처리하는 함수:
    1. '제3자' 형태의 단어를 임시 토큰으로 보호/복원하여 숫자를 보존.
    2. 불필요한 한글 이외의 문자를 제거.
    3. 형태소 분석 및 어간 추출.
    4. 불용어 및 1글자 단어 제거.
    """
    # 1단계: 한글, 공백을 제외한 모든 문자 제거
    # re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)는 한글과 공백만 남기고 나머지는 지우라는 의미
    # 제숫자 + 한글 글자 + 선택적 조사까지 포함
    # 딕셔너리를 함수 내부에 선언하여 매 호출마다 초기화 (핵심 수정 사항)
    protected_matches = {}

    def protect_term(match):
        # 찾은 문자열(예: '제3자')을 고유한 토큰(예: '###TOKEN0###')으로 매핑
        token = f"###TOKEN{len(protected_matches)}###"
        protected_matches[token] = match.group(0)
        return match.group(0) # 일단 원본 단어 그대로 둔 채로 형태소 분석 진행
    def strip_josa(text):
    # 한글 명사 뒤 “의”, “가”, “을” 등 제거
        return re.sub(r'([가-힣]+)(의|가|를|은|는|과|와|에|로|으로)$', r'\1', text)

    # 1단계: '제3자' 형태의 단어를 임시 토큰으로 치환하여 보호
    text = re.sub(r'(제\d+[가-힣]+)', protect_term, text)
    # 2단계: 한글, 공백, 보호 토큰의 문자(#)만 남기고 제거
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣# ]', '', text)

    # 2단계: Okt 형태소 분석기를 이용한 토큰화 및 어간 추출(Stemming)
    # okt.pos(text, stem=True)는 문장을 (단어, 품사) 형태로 나누고, '하다', '먹다'처럼 원형으로 만들어줍니다.
    # 예: "먹었었고" -> ('먹다', 'Verb')
    # Okt 형태소 분석기 객체 생성
    okt = Okt() 
    word_tokens = okt.pos(text, stem=True)

    # 3단계: 불용어 제거
    # 형태소 분석 후 의미 있는 단어만 추출하는 필터링 과정
    # 의미를 가지는 명사, 동사, 형용사, 부사 중에서 1글자 이상인 단어만 추출합니다.
    # 품사 태그가 'Josa'(조사), 'Eomi'(어미), 'Punctuation'(구두점) 등인 단어들을 제거합니다.
    meaningful_words = []
    for word, pos in word_tokens:
        if word in LEGAL_KEYWORDS:
            meaningful_words.append(word)
        elif pos in ['Noun','Verb'] and len(word) > 1:
            token = strip_josa(word)
            if token not in MINIMAL_STOPWORDS:
                meaningful_words.append(token)
     # 6. 보호된 단어 강제 포함 및 복원 (핵심)
    # 보호 목록에 있던 단어들을 강제로 최종 리스트에 추가합니다.
    # (이미 리스트에 분해되어 들어갔을 수 있지만, 완벽한 복원을 위해 강제 추가)
    for original_term in protected_matches.values():
        # '제3자' 자체를 하나의 단어로 명시적으로 추가
        meaningful_words.append(original_term)
        
    # 7. 중복 제거 및 최종 문자열 반환
    final_words = list(dict.fromkeys(meaningful_words))
    # 최종적으로 공백으로 연결된 문자열을 반환합니다. 
    # 모델에 따라 리스트 형태(meaningful_words)를 그대로 사용할 수도 있습니다.
    # 텍스트에서 분석에 의미 있는 핵심 단어들만 남긴 리스트 생성
    return ' '.join(final_words)

# 각 목적에 맞게 전처리 컬럼 생성 ---
# 질문 의도 파악 모델용: 'input' 컬럼만 전처리
# 'input' 컬럼에 전처리 함수를 적용하여 새로운 'input_processed' 컬럼 생성
df['input_processed'] = df['input'].apply(preprocess_text)

# 결과 비교를 위해 원본과 처리된 결과를 나란히 출력
print("\n=== 전처리 전/후 비교 ===")
for i in range(5):
    print(f"원본 [{i}]: {df['input'].iloc[i]}")
    print(f"결과 [{i}]: {df['input_processed'].iloc[i]}\n")


# 전처리된 데이터가 포함된 데이터프레임 확인
print("=== 각 목적에 맞게 생성된 전처리 컬럼들 ===")
print(df[['input', 'input_processed']].head())


=== 전처리 전/후 비교 ===
원본 [0]: 상고에서 상고이유서를 제출하는 기간의 법적 효력은 어떻게 됩니까?
결과 [0]: 상고 유서 제출 기간 법적

원본 [1]: 정신적 고통에 대한 손해배상은 어떤 경우에 인정되나요?
결과 [1]: 정신 고통 손해배상

원본 [2]: 양도소득세 납부와 관련한 채무불이행이 인정되는 경우, 피해자는 몇 가지 손해를 청구할 수 있나요?
결과 [2]: 양도소득세 납부 채무불이행 피해자 손해 청구

원본 [3]: 퇴직금 지급을 위한 근로자의 사직 의사표시가 유효하기 위한 조건은 무엇인가요?
결과 [3]: 퇴직금 지급 근로자 사직 의사표시

원본 [4]: 퇴직일이 실제 근로 종료일과 다른 경우 퇴직금 청구에 어떤 영향을 미치나요?
결과 [4]: 퇴직 일이 실제 근 종료 다른 퇴직금 청구 영향 미치나

=== 각 목적에 맞게 생성된 전처리 컬럼들 ===
                                               input  \
0               상고에서 상고이유서를 제출하는 기간의 법적 효력은 어떻게 됩니까?   
1                     정신적 고통에 대한 손해배상은 어떤 경우에 인정되나요?   
2  양도소득세 납부와 관련한 채무불이행이 인정되는 경우, 피해자는 몇 가지 손해를 청구...   
3        퇴직금 지급을 위한 근로자의 사직 의사표시가 유효하기 위한 조건은 무엇인가요?   
4         퇴직일이 실제 근로 종료일과 다른 경우 퇴직금 청구에 어떤 영향을 미치나요?   

                  input_processed  
0                  상고 유서 제출 기간 법적  
1                      정신 고통 손해배상  
2        양도소득세 납부 채무불이행 피해자 손해 청구  
3              퇴직금 지급 근로자 사직 의사표시  
4  퇴직 일이 실제 근 종료 다른 퇴직금 청구 영향 미치나  


In [28]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # 랜덤 셔플링

#  학습/테스트 데이터 분할

In [14]:
# 3. 학습/테스트 데이터 분할
def split_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    print(f"\n학습 데이터: {len(X_train)}개")
    print(f"테스트 데이터: {len(X_test)}개")
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(X, y)


학습 데이터: 3247개
테스트 데이터: 812개


# 토크나이저 로드

# 커스텀 데이터셋 정의
DivorceDataset 클래스는 PyTorch의 기본 Dataset 클래스를 상속받아 텍스트 데이터와 레이블(정답)을 모델이 학습할 수 있는 형태로 변환하고 제공하는 역할

In [10]:
class DivorceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts  #입력 텍스트 데이터 목록 (예: 문장 리스트)
        self.labels = labels #각 텍스트에 해당하는 정답 레이블 목록 (예: 0 또는 1)
        self.tokenizer = tokenizer  # 텍스트를 모델이 이해하는 숫자로 변환하는 데 사용되는 토크나이저 객체
        self.max_len = max_len
    
    def __len__(self):  # DataLoader가 데이터셋의 크기를 파악하는 데 사용
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        enc = self.tokenizer(  # 저장된 토크나이저를 사용하여 텍스트를 전처리
            text,
            max_length=self.max_len,  # 텍스트 길이를 max_len으로 제
            truncation=True,  # max_len을 초과하는 텍스트는 잘라냄
            padding='max_length', # max_len보다 짧은 텍스트는 부족한 길이를 0으로 채워 길이를 맞춤
            return_tensors='pt'  # 결과를 PyTorch 텐서(tensor) 형태로 반환하도록 지정
        )
        return {  # 모델 학습에 필요한 다음 세 가지 요소가 딕셔너리 형태로 반환
            'input_ids': enc['input_ids'].squeeze(),  # 텍스트의 각 단어/토큰이 매핑된 고유한 정수 ID 시퀀스
            'attention_mask': enc['attention_mask'].squeeze(),  # 시퀀스에서 실제 데이터(토큰)와 패딩(채워 넣은 0)을 구분하기 위한 마스크. (실제 토큰은 1, 패딩 토큰은 0)
            'labels': torch.tensor(label, dtype=torch.long) # 해당 텍스트의 정답 레이블 (PyTorch 롱 텐서 형태로 변환)
        }

# Dataset & DataLoader 생성
머신러닝 모델 학습을 효율적이고 정확하게 수행하기 위함

In [11]:
#  PyTorch와 BERT/KoBERT 모델이 요구하는 표준화된 형식으로 변환하고 캡슐화
train_dataset = DivorceDataset(X_train,y_train, tokenizer)
test_dataset = DivorceDataset(X_test,  y_test, tokenizer)

# DataLoader는 생성된 Dataset 객체로부터 데이터를 불러와 모델 학습에 적합한 형태로 관리하고 공급하는 핵심 도구
# batch_size=16 메모리 제한: 전체 데이터를 한 번에 메모리에 올리는 것을 방지하고, 학습 안정성을 높임
# shuffle=True 편향 방지: 데이터의 특정 순서(예: 긍정 데이터가 먼저 나오고 부정 데이터가 나중에 나오는 경우)에 모델이 익숙해지는 것을 방지하여 일반화 성능을 높임
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# KoBERT 모델 로드
BertModel : 입력 텍스트를 이해하고 특징을 추출하는 코어 언어 모델
encoder :   BERT 모델의 핵심 부분으로, 텍스트의 문맥적 특징을 추출
pooler : 인코더의 최종 출력을 분류 작업에 사용할 수 있는 고정된 크기의 벡터로 압축

분류를 위한 헤드: (classifier) 

# PyTorch에서 BERT 모델을 fine-tuning(학습)

# Optimizer 설정
모델이 데이터에서 학습할 수 있도록 방향을 잡아주는 역할
lr=2e-5 → 학습률, 너무 크면 발산, 너무 작으면 느리게 수렴
AdamW: 모델을 훈련시키는 데 사용되는 최적화 알고리즘 => Transformer 모델에서 흔히 발생하는 가중치 감소(Weight Decay) 문제를 더 잘 처리하도록 설계됨 
AdamW 옵티마이저가 2e−5의 속도로 KoBERT 모델의 모든 가중치를 수정하여 최적의 성능을 찾도록 준비하는 단계

# 학습 루프

In [29]:
"""
KoBERT 기반 이혼 vs 비이혼 분류 모델
- 문맥 이해 능력 우수
- 사전학습된 한국어 BERT 모델 활용
- Fine-tuning으로 도메인 적응

필요한 라이브러리 설치:
pip install torch transformers
pip install kobert-transformers
pip install scikit-learn pandas numpy matplotlib seaborn
"""

# ========================================
# 1. 설정 및 하이퍼파라미터
# ========================================
class Config:
    # 모델 설정
    MODEL_NAME = 'monologg/kobert'  # KoBERT 사전학습 모델
    MAX_LEN = 128  # 최대 토큰 길이
    BATCH_SIZE = 16  # 배치 크기 (GPU 메모리에 따라 조정)
    EPOCHS = 3  # 학습 에폭 (3-5 추천)
    LEARNING_RATE = 2e-5  # 학습률
    DROPOUT_RATE = 0.3  # 드롭아웃 비율
    
    # 데이터 설정
    TEST_SIZE = 0.2
    VAL_SIZE = 0.1  # Validation set 비율
    RANDOM_STATE = 42
    
    # 기타
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SAVE_PATH = 'kobert_divorce_model.pt'

config = Config()
print(f"사용 디바이스: {config.DEVICE}")
print(f"GPU 사용 가능: {torch.cuda.is_available()}")

# ========================================
# 2. 데이터셋 클래스
# ========================================
class DivorceDataset(Dataset):
    """
    이혼/비이혼 분류를 위한 커스텀 데이터셋
    """
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # 토크나이저로 인코딩
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # [CLS], [SEP] 추가
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# ========================================
# 3. KoBERT 분류 모델
# ========================================
class KoBERTClassifier(nn.Module):
    """
    KoBERT + 분류 헤드
    """
    def __init__(self, n_classes=2, dropout=0.3):
        super(KoBERTClassifier, self).__init__()
        
        # KoBERT 모델 로드
        self.bert = BertModel.from_pretrained('monologg/kobert', trust_remote_code=True)
        
        # 분류 헤드
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        # BERT 인코딩
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # [CLS] 토큰의 hidden state 사용
        pooled_output = outputs.pooler_output
        
        # 드롭아웃 + 분류
        output = self.dropout(pooled_output)
        logits = self.classifier(output)
        
        return logits

# ========================================
# 4. 데이터 준비 함수
# ========================================
def prepare_data(df):
    """
    데이터 로드 및 전처리
    """
    print("\n=== 데이터 준비 ===")
    
    # 텍스트와 레이블 추출
    texts = df['input_processed'].values  # 질의 전처리 데이터
    labels = df['is_divorce'].values # 0 또는 1

    
    # 데이터 분포 확인
    print(f"전체 데이터: {len(df)}개")
    print(f"이혼: {sum(labels == 1)}개 ({sum(labels == 1)/len(labels)*100:.1f}%)")
    print(f"비이혼: {sum(labels == 0)}개 ({sum(labels == 0)/len(labels)*100:.1f}%)")
    
    return texts, labels

def create_data_loaders(texts, labels, tokenizer):
    """
    Train/Val/Test 데이터로더 생성
    """
    # Train/Test 분할
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        texts, labels,
        test_size=config.TEST_SIZE,
        random_state=config.RANDOM_STATE,
        stratify=labels
    )
    
    # Train/Val 분할
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_texts, train_labels,
        test_size=config.VAL_SIZE,
        random_state=config.RANDOM_STATE,
        stratify=train_labels
    )
    
    print(f"\n학습 데이터: {len(train_texts)}개")
    print(f"검증 데이터: {len(val_texts)}개")
    print(f"테스트 데이터: {len(test_texts)}개")
    
    # 데이터셋 생성
    train_dataset = DivorceDataset(train_texts, train_labels, tokenizer, config.MAX_LEN)
    val_dataset = DivorceDataset(val_texts, val_labels, tokenizer, config.MAX_LEN)
    test_dataset = DivorceDataset(test_texts, test_labels, tokenizer, config.MAX_LEN)
    
    # 데이터로더 생성
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)
    
    return train_loader, val_loader, test_loader

# ========================================
# 5. 학습 함수
# ========================================
def train_epoch(model, data_loader, optimizer, scheduler, device):
    """
    1 에폭 학습
    """
    model.train()
    losses = []
    correct_predictions = 0
    total_samples = 0
    
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        logits = model(input_ids, attention_mask)
        
        # Loss 계산 (CrossEntropyLoss에 class_weight 적용 가능)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping (폭발 방지)
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        scheduler.step()
        
        # 통계
        losses.append(loss.item())
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_samples += labels.size(0)
    
    return correct_predictions.double() / total_samples, np.mean(losses)

def eval_model(model, data_loader, device):
    """
    모델 평가
    """
    model.eval()
    losses = []
    correct_predictions = 0
    total_samples = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            logits = model(input_ids, attention_mask)
            
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            
            losses.append(loss.item())
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            total_samples += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = correct_predictions.double() / total_samples
    return accuracy, np.mean(losses), all_preds, all_labels

# ========================================
# 6. 전체 학습 파이프라인
# ========================================
def train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs):
    """
    전체 학습 루프
    """
    best_accuracy = 0
    history = {'train_acc': [], 'train_loss': [], 'val_acc': [], 'val_loss': []}
    
    for epoch in range(epochs):
        print(f'\n=== Epoch {epoch + 1}/{epochs} ===')
        
        # 학습
        train_acc, train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
        
        # 검증
        val_acc, val_loss, _, _ = eval_model(model, val_loader, device)
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
        
        # 히스토리 저장
        history['train_acc'].append(train_acc.item())
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc.item())
        history['val_loss'].append(val_loss)
        
        # 최고 모델 저장
        if val_acc > best_accuracy:
            torch.save(model.state_dict(), config.SAVE_PATH)
            best_accuracy = val_acc
            print(f'✅ 모델 저장! (Val Acc: {val_acc:.4f})')
    
    return history

# ========================================
# 7. 평가 및 시각화
# ========================================
def evaluate_and_visualize(model, test_loader, device):
    """
    테스트 데이터 평가 및 결과 시각화
    """
    print("\n=== 테스트 데이터 평가 ===")
    
    # 최고 모델 로드
    model.load_state_dict(torch.load(config.SAVE_PATH))
    
    test_acc, test_loss, y_pred, y_true = eval_model(model, test_loader, device)
    
    print(f'Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'F1 Score: {f1_score(y_true, y_pred):.4f}')
    
    # 분류 리포트
    print("\n분류 리포트:")
    print(classification_report(y_true, y_pred, target_names=['비이혼', '이혼'], digits=4))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['비이혼', '이혼'],
                yticklabels=['비이혼', '이혼'])
    plt.title('Confusion Matrix - KoBERT')
    plt.ylabel('실제')
    plt.xlabel('예측')
    plt.tight_layout()
    plt.savefig('kobert_confusion_matrix.png', dpi=300)
    print("\n✅ Confusion Matrix 저장: kobert_confusion_matrix.png")
    
    return y_pred, y_true

def plot_training_history(history):
    """
    학습 히스토리 시각화
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    ax1.plot(history['train_acc'], label='Train Accuracy')
    ax1.plot(history['val_acc'], label='Val Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Loss
    ax2.plot(history['train_loss'], label='Train Loss')
    ax2.plot(history['val_loss'], label='Val Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('kobert_training_history.png', dpi=300)
    print("✅ 학습 히스토리 저장: kobert_training_history.png")

# ========================================
# 8. 예측 함수
# ========================================
def predict_text(model, tokenizer, text, device):
    """
    새로운 텍스트 예측
    """
    model.eval()
    
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=config.MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1)
    
    result = "이혼" if pred.item() == 1 else "비이혼"
    confidence = probs[0][pred.item()].item() * 100
    
    print(f"\n예측 결과: {result}")
    print(f"신뢰도: {confidence:.2f}%")
    print(f"확률 - 비이혼: {probs[0][0].item():.4f}, 이혼: {probs[0][1].item():.4f}")
    
    return pred.item(), probs[0].cpu().numpy()

# ========================================
# 9. 메인 실행 함수
# ========================================
def run_kobert_pipeline(df):
    """
    KoBERT 전체 파이프라인 실행
    """
    print("=" * 50)
    print("KoBERT 이혼 vs 비이혼 분류 모델 학습 시작")
    print("=" * 50)
    
    # 1. 데이터 준비
    texts, labels = prepare_data(df)
    
    # 2. 토크나이저 로드
    print("\n토크나이저 로드 중...")
    tokenizer = AutoTokenizer.from_pretrained('monologg/kobert',  trust_remote_code=True)
   
    # 3. 데이터로더 생성
    train_loader, val_loader, test_loader = create_data_loaders(texts, labels, tokenizer)
    
    # 4. 모델 초기화
    print("\n모델 초기화 중...")
    model = KoBERTClassifier(n_classes=2, dropout=config.DROPOUT_RATE)
    model = model.to(config.DEVICE)
    
    # 5. Optimizer & Scheduler 설정
    optimizer = AdamW(model.parameters(), lr=config.LEARNING_RATE)
    total_steps = len(train_loader) * config.EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_steps * 0.1),
        num_training_steps=total_steps
    )
    
    # 6. 모델 학습
    history = train_model(
        model, train_loader, val_loader, 
        optimizer, scheduler, config.DEVICE, config.EPOCHS
    )
    
    # 7. 학습 히스토리 시각화
    plot_training_history(history)
    
    # 8. 테스트 평가
    y_pred, y_true = evaluate_and_visualize(model, test_loader, config.DEVICE)
    
    print("\n" + "=" * 50)
    print("학습 완료!")
    print("=" * 50)
    
    return model, tokenizer

# ========================================
# 10. 사용 예시
# ========================================

# 데이터 로드
# df = pd.read_csv('divorce_data.csv')
# # 필요 컬럼: 'text', 'is_divorce'

# 모델 학습
model, tokenizer = run_kobert_pipeline(df)

# 새로운 텍스트 예측
test_texts = [
    "남편이 날 때려서 헤어지고 싶어",
    "이혼 소송을 진행하고 싶습니다",
    "계약서 작성 문의드립니다"
]

for text in test_texts:
    print(f"\n입력: {text}")
    predict_text(model, tokenizer, text, config.DEVICE)

# 모델 저장 (이미 학습 중 저장됨)
# torch.save(model.state_dict(), 'kobert_divorce_final.pt')

# 모델 로드
# model.load_state_dict(torch.load('kobert_divorce_final.pt'))


사용 디바이스: cpu
GPU 사용 가능: False
KoBERT 이혼 vs 비이혼 분류 모델 학습 시작

=== 데이터 준비 ===
전체 데이터: 4071개
이혼: 721개 (17.7%)
비이혼: 3350개 (82.3%)

토크나이저 로드 중...

학습 데이터: 2930개
검증 데이터: 326개
테스트 데이터: 815개

모델 초기화 중...

=== Epoch 1/3 ===


Training:   0%|          | 0/184 [00:00<?, ?it/s]

Train Loss: 0.2626 | Train Acc: 0.9096


Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

Val Loss: 0.0889 | Val Acc: 0.9816
✅ 모델 저장! (Val Acc: 0.9816)

=== Epoch 2/3 ===


Training:   0%|          | 0/184 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
torch.save(model.state_dict(), "kobert_divorce_parttt.pt")

In [ ]:
torch.save(model.state_dict(), "kobert_divorce_final.pt")
# 모델 로드
model.load_state_dict(torch.load("kobert_divorce_final.pt"))ㅇ
model.to(device)

# optimizer도 그대로 이어서 설정 가능
optimizer = AdamW(model.parameters(), lr=2e-5)

# 다시 학습 루프 실행